In [17]:
!pip install ultralytics
!pip install opencv-python-headless
!pip install albumentations
!pip install labelImg

In [24]:
import torch
from ultralytics import YOLO

In [25]:
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

CUDA: True
GPU: NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [19]:
datasets = [
    {
        "name": "dietary_app_segment",
        "path": "/home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8"
    },
    {
        "name": "food_calories",
        "path": "/home/muliaandiki/project/NutriPlate/preprocessing/data/Food Calories.v3i.yolov8"
    },
    {
        "name": "nutritionverse",
        "path": "/home/muliaandiki/project/NutriPlate/preprocessing/data/Nutritionverse"
    }
]

In [20]:
print(datasets)

[{'name': 'dietary_app_segment', 'path': '/home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8'}]


In [21]:
# Load Model
base_model = "yolov8m.pt"

In [22]:
for ds in datasets:
    print(f"🚀 Training dataset: {ds['name']}")
    model = YOLO(base_model)
    
    model.train(
        data=f"{ds['path']}/data.yaml",
        epochs=100,
        imgsz=640,
        batch=1,
        project="runs/train",
        name=ds["name"],
        exist_ok=True,
        device=0,
        optimizer="AdamW",
        patience=20,
        verbose=True,
        amp=True,
    )
    
    print(f" Finished training {ds['name']}\n")

🚀 Training dataset: dietary_app_segment
New https://pypi.org/project/ultralytics/8.3.237 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.236 🚀 Python-3.12.3 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 5804MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=30

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 5.67 GiB of which 18.06 MiB is free. Including non-PyTorch memory, this process has 5.62 GiB memory in use. Of the allocated memory 5.44 GiB is allocated by PyTorch, and 28.92 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)